In [2]:
import cvxpy as cp

# Define variables
c1 = cp.Variable()
c2 = cp.Variable()
nl = cp.Variable()
nm = cp.Variable()
nh = cp.Variable()
ol = cp.Variable()
oh = cp.Variable()
r = cp.Variable()
gr = cp.Variable()
oc = cp.Variable()
gc = cp.Variable()
Pp = cp.Variable()
Pr = cp.Variable()
Jf = cp.Variable()
Of = cp.Variable()
Ol = cp.Variable()

# Define constraints
constraints = [
    c1 <= 20000,
    c2 <= 30000,
    c1 + c2 <= 45000,
    nl + nm + nh <= 10000,
    ol + oh <= 8000,
    500 <= Ol,
    Ol <= 1000,
    Pr >= 0.40 * Pp,
    nl == 0.1 * c1 + 0.15 * c2,
    nm == 0.2 * c1 + 0.25 * c2,
    nh == 0.2 * c1 + 0.18 * c2,
    ol == 0.12 * c1 + 0.08 * c2,
    oh == 0.2 * c1 + 0.19 * c2,
    r == 0.13 * c1 + 0.12 * c2,
    gr == 0.6 * nl + 0.52 * nm + 0.45 * nh,
    oc == 0.68 * ol + 0.75 * oh,
    gc == 0.28 * ol + 0.2 * oh,
    Jf <= 1.0 * ol + 0.6 * oh + 1.5 * oc + 0.05 * r,
    Pp >= 84 * (nl + nm + nh + gr + gc),
    Pr >= 94 * (nl + nm + nh + gr + gc)
]

# Define objective function
profit = 700 * Pp + 600 * Pr + 400 * Jf + 350 * Of + 150 * Ol

# Define problem
problem = cp.Problem(cp.Maximize(profit), constraints)

# Solve problem
problem.solve()

# Print results
print(f"Optimal profit: {problem.value}")
print(f"c1: {c1.value}")
print(f"c2: {c2.value}")
print(f"nl: {nl.value}")
print(f"nm: {nm.value}")
print(f"nh: {nh.value}")
print(f"ol: {ol.value}")
print(f"oh: {oh.value}")
print(f"r: {r.value}")
print(f"gr: {gr.value}")
print(f"oc: {oc.value}")
print(f"gc: {gc.value}")
print(f"Pp: {Pp.value}")
print(f"Pr: {Pr.value}")
print(f"Jf: {Jf.value}")
print(f"Of: {Of.value}")
print(f"Ol: {Ol.value}")

Optimal profit: inf
c1: None
c2: None
nl: None
nm: None
nh: None
ol: None
oh: None
r: None
gr: None
oc: None
gc: None
Pp: None
Pr: None
Jf: None
Of: None
Ol: None


In [ ]:
import numpy as np

def print_tableau(tableau, basis, iteration):
    """
    Pretty prints the current tableau for debugging.
    
    Parameters:
    tableau (numpy.ndarray): The current tableau.
    basis (list): The current basis variables.
    iteration (int): The current iteration number.
    """
    print(f"\nIteration {iteration}")
    print("-" * 40)
    print("Basis  | ", end="")
    for i in range(tableau.shape[1] - 1):
        print(f"x{i: <3}", end="  ")
    print("RHS")

    for i, row in enumerate(tableau):
        print(f"x{basis[i]: <5} | ", end="")
        for val in row:
            print(f"{val:7.2f}", end=" ")
        print()
    print("-" * 40)

def initial_simplex(c, A, b):
    """
    Initialize the simplex algorithm by finding a basic feasible solution.

    Parameters:
    c (numpy.ndarray): Coefficients of the objective function.
    A (numpy.ndarray): Coefficients of the constraints.
    b (numpy.ndarray): Right-hand side of the constraints.

    Returns:
    tuple: Initial basis, initial tableau, and updated cost vector.
    """
    m, n = A.shape

    # Add slack variables to convert inequalities to equalities
    A = np.hstack([A, np.eye(m)])
    c = np.hstack([c, np.zeros(m)])

    # Initial basis (slack variables start in the basis)
    basis = list(range(n, n + m))

    # Initial tableau: [A | b]
    tableau = np.hstack([A, b.reshape(-1, 1)])
    print_tableau(tableau, basis, 'start')
    return basis, tableau, c, A  # Return extended A as well

def pivot(tableau, basis, entering, leaving):
    """
    Perform a pivot operation in the simplex algorithm.

    Parameters:
    tableau (numpy.ndarray): The current tableau.
    basis (list): The current basis.
    entering (int): The index of the entering variable.
    leaving (int): The index of the leaving variable.

    Returns:
    tuple: Updated tableau and updated basis.
    """
    m, _ = tableau.shape

    # Normalize the pivot row
    pivot_row = tableau[leaving, :]
    pivot_element = pivot_row[entering]
    tableau[leaving, :] = pivot_row / pivot_element

    # Update the other rows
    for i in range(m):
        if i != leaving:
            tableau[i, :] -= tableau[i, entering] * tableau[leaving, :]

    # Update the basis
    basis[leaving] = entering

    return tableau, basis

def simplex(c, A, b):
    """
    Simplex algorithm to solve the linear programming problem:
    maximize c^T x
    subject to Ax <= b, x >= 0

    Parameters:
    c (numpy.ndarray): Coefficients of the objective function.
    A (numpy.ndarray): Coefficients of the constraints.
    b (numpy.ndarray): Right-hand side of the constraints.

    Returns:
    numpy.ndarray: Optimal solution vector x.
    float: Optimal value of the objective function.
    """
    basis, tableau, c, A_ext = initial_simplex(c, A, b)
    m, n = A.shape
    num_vars = n + m  # Total number of variables including slack variables

    iteration = 0
    print_tableau(tableau, basis, iteration)

    while True:
        # Compute the reduced costs
        cb = c[basis]
        B = tableau[:, basis]  # Extract the basis matrix from the tableau
        lam = np.linalg.solve(B.T, cb)  # Dual variables
        reduced_costs = c - A_ext.T @ lam  # Use A_ext (with slack variables)

        # Check for optimality
        if np.all(reduced_costs >= 0):
            x = np.zeros(num_vars)
            x[basis] = tableau[:, -1]  # Assign values to basic variables
            print("\nOptimal solution found!")
            return x[:n], np.dot(c[:n], x[:n])  # Only use original variables in objective

        # Determine entering variable
        entering = np.argmin(reduced_costs)

        # Determine leaving variable
        d = np.linalg.solve(B, A_ext[:, entering])
        if np.all(d <= 0):
            raise ValueError("Linear program is unbounded.")

        # Compute ratios for minimum ratio test
        valid_ratios = np.where(d > 0, tableau[:, -1] / d, np.inf)
        leaving = np.argmin(valid_ratios)

        print(f"\nPivot: Entering x{entering}, Leaving x{basis[leaving]}")
        
        # Perform pivot operation
        tableau, basis = pivot(tableau, basis, entering, leaving)
        iteration += 1
        print_tableau(tableau, basis, iteration)

# Example usage
A = np.array([[5, 2], [-3, 2], [-2, -3]])
b = np.array([31, 5, -1])
c = np.array([-9, -4])

optimal_solution, optimal_value = simplex(c, A, b)
print("\nOptimal solution:", optimal_solution)
print("Optimal value:", optimal_value)



Iteration start
----------------------------------------
Basis  | x0    x1    x2    x3    x4    RHS
x2     |    5.00    2.00    1.00    0.00    0.00   31.00 
x3     |   -3.00    2.00    0.00    1.00    0.00    5.00 
x4     |   -2.00   -3.00    0.00    0.00    1.00   -1.00 
----------------------------------------

Iteration 0
----------------------------------------
Basis  | x0    x1    x2    x3    x4    RHS
x2     |    5.00    2.00    1.00    0.00    0.00   31.00 
x3     |   -3.00    2.00    0.00    1.00    0.00    5.00 
x4     |   -2.00   -3.00    0.00    0.00    1.00   -1.00 
----------------------------------------

Pivot: Entering x0, Leaving x2

Iteration 1
----------------------------------------
Basis  | x0    x1    x2    x3    x4    RHS
x0     |    1.00    0.40    0.20    0.00    0.00    6.20 
x3     |    0.00    3.20    0.60    1.00    0.00   23.60 
x4     |    0.00   -2.20    0.40    0.00    1.00   11.40 
----------------------------------------

Optimal solution found!

Op